In [82]:
from gfootball.env.wrappers import CheckpointRewardWrapper

def reward_modified(self,reward):
    '''
    o['active'] => 1 or 2 
    '''
    #change checkpoints to only 4
    #self._num_checkpoints = 4
    
    #assuming max of 500 steps 
    passing_rewards = 0.002 
    shoting_rewards = 0.1
    not_owning_ball_team = -0.001
    owning_ball = 0.001
    observation = self.env.unwrapped.observation()
    actions = self.env.unwrapped._get_actions()
    #print(observation)
    if observation is None:
        return reward

    assert len(reward) == len(observation)

    for rew_index in range(len(reward)):
        o = observation[rew_index]
        a = actions[rew_index]
        if reward[rew_index] == 1: #scoring a goal
            reward[rew_index] += self._checkpoint_reward * (
                self._num_checkpoints -
                self._collected_checkpoints.get(rew_index, 0))
            self._collected_checkpoints[rew_index] = self._num_checkpoints
            #add extra points to compensate for the extra postive rewards
            reward[rew_index] += 0.5
            continue
        
        
        #check if the right team does own the ball, this should be the first objective in the game
        if o['ball_owned_team'] != 0:
            #reward[rew_index] += not_owning_ball_team
            #player need to get closer to get the ball 
            player_pos = o['right_team'][o['active']]
            player_ball_dist = ((o['ball'][0] - player_pos[0]) ** 2 + (o['ball'][1]-player_pos[1]) ** 2) ** 0.5
            reward[rew_index] += player_ball_dist*not_owning_ball_team
        else:
            #they should positive reward for owning the ball too
            reward[rew_index] += owning_ball
            
        
        
        # Check if the active player has the ball.
        if ('ball_owned_team' not in o or
          o['ball_owned_team'] != 0 or
          'ball_owned_player' not in o or
          o['ball_owned_player'] != o['active']):
            continue
        #now, we know that the active player has the ball
        #reward passing if the second player is closer to goal
        #reward shooting if active player is near the goal
        passing_thre = 0.05
        position_second_player = o['right_team'][1 if o['active']==2 else 2]
        dist_second_player = ((position_second_player[0] - 1) ** 2 + position_second_player[1] ** 2) ** 0.5
        d = ((o['ball'][0] - 1) ** 2 + o['ball'][1] ** 2) ** 0.5 #ball dist to goal (the active player too)
        player_distances = ((o['ball'][0] - position_second_player[0]) ** 2 + (o['ball'][1]-position_second_player[1]) ** 2) ** 0.5
        #if the ball is very close, shooting would be better than passing
        if dist_second_player + passing_thre < d and d > 0.3: 
            if player_distances < 0.2 and a == 11: #short pass when they are near each other
                reward[rew_index] += passing_rewards
            elif player_distances > 0.2 and (a == 9 or a==10):
                reward[rew_index] += passing_rewards
        #now check if the ball is near target, then rewarding shotting 
        if d < 0.4 and a ==12:
            reward[rew_index] += shoting_rewards
            

        # Collect the checkpoints.
        # We give reward for distance 1 to 0.2.
        while (self._collected_checkpoints.get(rew_index, 0) <
             self._num_checkpoints):
            if self._num_checkpoints == 1:
                threshold = 0.99 - 0.8
            else:
                threshold = (0.99 - 0.8 / (self._num_checkpoints - 1) *
                           self._collected_checkpoints.get(rew_index, 0))
            if d > threshold:
                break
            reward[rew_index] += self._checkpoint_reward
            self._collected_checkpoints[rew_index] = (
                self._collected_checkpoints.get(rew_index, 0) + 1)
    return reward

CheckpointRewardWrapper.reward = reward_modified


In [83]:
from rldm.utils import football_tools

In [84]:
env = football_tools.RllibGFootball('3_vs_3_auto_GK')

In [85]:
football_tools.n_players_to_env_name(3, True)

'3_vs_3_auto_GK'

In [86]:
#def rollout(num_episodes:int):
num_episodes =1
env = football_tools.RllibGFootball('3_vs_3_auto_GK')
once = False
for i in range(num_episodes):
    observations = env.reset()
#         print(observations['player_0'][36:])
#         print(observations['player_1'][36:])
    done, ep_reward = False, {a: 0 for a in observations}
    while not done:
        random_actions = {a: env.action_space[a].sample() for a in env.action_space}
        observations, rewards, dones, infos = env.step(random_actions)
        print(rewards)
        if once:
            observations, rewards, dones, infos = env.step(random_actions)
            #print(infos['player_0']['game_info']['active'])
            #print(rewards)
            break
    
        #print(observations['player_0'][36:])
        #print(observations['player_1'][36:])
        ep_reward = {a: ep_reward[a] + rewards[a] for a in rewards}
        done = dones['__all__']
    print(f"Episode {i} completed.")
    print("Rewards:")
    for a, ep_r in ep_reward.items():
        print(f"\t{a}: {ep_r}")
    print(f"\ttotal: {sum(ep_reward.values())}\n")

{'player_0': -0.2, 'player_1': -0.2}
{'player_0': -0.2, 'player_1': -0.2}
{'player_0': -0.2, 'player_1': -0.2}
{'player_0': -0.2, 'player_1': -0.2}
{'player_0': 0.0, 'player_1': 0.0}
{'player_0': 0.0, 'player_1': 0.0}
{'player_0': 0.0, 'player_1': 0.02}
{'player_0': 0.0, 'player_1': 0.0}
{'player_0': 0.0, 'player_1': 0.02}
{'player_0': 0.0, 'player_1': 0.0}
{'player_0': -0.2, 'player_1': -0.2}
{'player_0': -0.2, 'player_1': -0.2}
{'player_0': -0.2, 'player_1': -0.2}
{'player_0': -0.2, 'player_1': -0.2}
{'player_0': -0.2, 'player_1': -0.2}
{'player_0': -0.2, 'player_1': -0.2}
{'player_0': -0.2, 'player_1': -0.2}
{'player_0': -0.2, 'player_1': -0.2}
{'player_0': -0.2, 'player_1': -0.2}
{'player_0': -0.2, 'player_1': -0.2}
{'player_0': -0.2, 'player_1': -0.2}
{'player_0': -0.2, 'player_1': -0.2}
{'player_0': -0.2, 'player_1': -0.2}
{'player_0': -0.2, 'player_1': -0.2}
{'player_0': -0.2, 'player_1': -0.2}
{'player_0': -0.2, 'player_1': -0.2}
{'player_0': -0.2, 'player_1': -0.2}
{'player_0'

In [ ]:
# rollout(1)

In [ ]:
'ball_owned_team' not in infos['player_0']

In [ ]:
(0.99 - 0.8 / (10 - 1) *0)

In [ ]:
(0.8**2 + 0.2 **2)**0.5

In [ ]:
0.922
0.825
